# Trenutno __ne__ dela

Uporabi ***ep_keras_rnn.py*** s pomočjo ***main.py***.

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
### parameters

NUM_STEPS = 128
NUM_PRED = NUM_STEPS // 4 
BATCH_SIZE = 5
print(NUM_PRED)

32


In [3]:
# A function that transforms raw data to an array of sequence data.
# The attribute raw_data should be a numpy 2-D array of float32 numbers.
# Each row represents a trade and has 2 columns, price (column 0) and amount (column 1)
def transform_to_seq(raw_data):
    # The output (y) is the prediction of bitcoin prices of the next few (#num_pred) trades 
    # For the training data, this is represented by the real next (num_pred) trade prices.
    
    # To get the most of our data, we remove the last #num_pred trade prices and use them as 
    # the labels of the last input x
    
    temp_y = raw_data[len(raw_data) - NUM_PRED:, 0]
    raw_data = raw_data[:len(raw_data) - NUM_PRED]
    

    # Remove the excess data from the start of the timeseries, if
    # the number of steps (num_steps) doesn't devide the size of the remaining data.
    
    if len(raw_data) % NUM_STEPS != 0:
        raw_data = raw_data[len(raw_data)%NUM_STEPS:]
        
    # Split the whole data into chunks, each the length of number of steps (num_steps).
    d = len(raw_data) // NUM_STEPS
    x = np.array(np.split(raw_data, d))
    
    # Gather the rest of the labels from x and append the last label from before.
    
    y = x[1:,:NUM_PRED, 0]
    y = np.insert(y, -1, temp_y, axis=0)
    
    # Finnaly we return the transformed sequential data as a dictionary, 
    # with input x and labels y
    return {"x":x, "y": y}

In [4]:
# A function that returns a random sample from the dictionary data (x: , y: )
# Each (x and y) should represent a numpy array, where the each entry in the 1st dim,
# represents an input or output vector.
def batcher(data):
    # Create a random mask of indexes. Attributes: max index: length of data,
    # number of indexes: batch_size, with no recurring indexes.
    mask = np.random.choice(range(len(data['x'])), BATCH_SIZE, replace=False)
    
    # Create the x and y batch of the same indexes.
    batch = {}
    batch['x'] = data['x'][mask]
    batch['y'] = data['y'][mask]
    
    return batch

In [5]:
rd = np.load('sample_data.npy')
data = transform_to_seq(rd[:,[1,2]])
print(batcher(data))

{'x': array([[[  1.05881000e+03,   6.37771000e-01],
        [  1.05752000e+03,   1.82890000e+00],
        [  1.05752000e+03,   1.53000000e-01],
        ..., 
        [  1.05114000e+03,   3.40951510e-01],
        [  1.05307000e+03,   5.82782500e-02],
        [  1.05134000e+03,   1.30234000e+00]],

       [[  1.05134000e+03,   8.29170000e-01],
        [  1.05133000e+03,   9.90000000e-02],
        [  1.05076000e+03,   1.65810000e-01],
        ..., 
        [  1.05774000e+03,   2.44900000e-02],
        [  1.05825000e+03,   2.40000000e-02],
        [  1.05854000e+03,   3.88165000e-02]],

       [[  1.06278000e+03,   1.00584950e-01],
        [  1.06390000e+03,   1.05890700e-01],
        [  1.06282000e+03,   3.00000000e-02],
        ..., 
        [  1.06498000e+03,   1.00000000e+00],
        [  1.06498000e+03,   6.10200000e-01],
        [  1.06498000e+03,   2.68000000e-02]],

       [[  1.05854000e+03,   4.21835000e-02],
        [  1.05910000e+03,   2.01149930e-01],
        [  1.05912000e+03,

In [6]:
# Define the model.
def RNN_model():
    
    x = tf.placeholder(tf.float32, [None, NUM_STEPS, 2], name='input_placeholder')
    y = tf.placeholder(tf.float32, [None, NUM_STEPS], name='labels_placeholder')

    lstm_cell = tf.contrib.rnn.BasicLSTMCell(NUM_STEPS)
    
    state = tf.zeros([BATCH_SIZE, lstm.state_size])
    
    outputs, states = tf.contrib.rnn.basic_rnn_seq2seq(x, y, lstm_cell, feed_previous=False)
    
    """
    Inputs
    """

    rnn_inputs = tf.one_hot(x, num_classes)

    """
    RNN
    """
    
    cell = tf.contrib.rnn.BasicRNNCell(state_size)
    rnn_outputs, final_state = tf.nn.dynamic_rnn(cell, rnn_inputs, initial_state=init_state)

    """
    Predictions, loss, training step
    """

    with tf.variable_scope('softmax'):
        W = tf.get_variable('W', [state_size, num_classes])
        b = tf.get_variable('b', [num_classes], initializer=tf.constant_initializer(0.0))
    logits = tf.reshape(
                tf.matmul(tf.reshape(rnn_outputs, [-1, state_size]), W) + b,
                [batch_size, num_steps, num_classes])
    predictions = tf.nn.softmax(logits)

    losses = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    total_loss = tf.reduce_mean(losses)
    train_step = tf.train.AdagradOptimizer(learning_rate).minimize(total_loss)

In [7]:
RNN_model()

NameError: name 'lstm' is not defined